# Metody Obliczeniowe w Nauce i Technice Laboratorium 4
## Singular Value Decomposition
### Błażej Kustra

In [6]:
import sys
# sys.path.append("/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages")

import numpy as np
from matplotlib import pyplot as plt
import os
import re
import io
import math
import scipy
from num2words import num2words

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import nltk

stop_words = set(stopwords.words('english'))

# import ssl

# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()

# Wyszukiwarka
### 1. Przygotuj duży (>1000 elementów) zbiór dokumentów tekstowych w języku angielskim (np. wybrany korpus tekstów, podzbiór artykułów Wikipedii, zbiór dokumentów HTML uzyskanych za pomocą Web-crawlera, zbiór rozdziałów wyciętych z różnych książek).

Pobrałem 1386 recenzji filmów opublikowanych na stronie imdb.com - http://www.cs.cornell.edu/people/pabo/movie-review-data/

### 2. Określ słownik słów kluczowych (termów) potrzebny do wyznaczenia wektorów cech bag-of-words (indeksacja). Przykładowo zbiorem takim może być unia wszystkich słów występujących we wszystkich tekstach.
Przygotowalem liste wszystkich słów + słownik słów razem z liczbą wystąpień każdego.
Dodatkowo przekonwertowałem wszystkie recenzje w paru krokach:
 - wszystkie znaki zamienione na "małe"
 - usunięte wszelkie znaki specjalne, razem z kropkami/przecinkami
 - usuniete powtarzające się spacje
 - skorzystałem z biblioteki "nltk":
  - usunięcie wszytskich stop_words ("the", "a", itp.)
  - użycie lemmatizera który formatuje słowa na pierwotne ( "playing" -> "play", "brothers" -> "brother", itp.)
 - przekonwertowanie liczb na wyrazy ("42" -> "fourty two", itp.)

In [7]:
def sort_dir(data):
    """
    Used for sorting like this: 1,2,3,4 ...
    not like: 1 10 100 1000 1001 2 20 200 2000 3 ... etc
    """
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [8]:
lemmatizer = WordNetLemmatizer() 

def preprocess(text):
    text = text.lower()
    text = text.replace("_","")
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub('  +',' ',text)

    text_list = text.split()
    text = ""
    
    for word in text_list:

        word = lemmatizer.lemmatize(word, pos="v")

        try: word = num2words(int(word))
        except ValueError: pass
        
        if word in stop_words: continue

        text += word + " "

    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'[0-9]','',text)
    
    return text


def file_preprocess(file):
    if file == ".DS_Store": 
        return None
            
    f = io.open('data/'+ file, encoding='windows-1254')
    text = f.read()
    f.close()
    
    return preprocess(text)

In [9]:
def make_dictionary():
    dictionary = {}
    files = sort_dir(os.listdir('data/'))
    for file in files:
        text = file_preprocess(file)
        if text is None: continue
            
        for word in text.split():
            if word in dictionary.keys():
                dictionary[word] += 1
            else:
                dictionary[word] = 1
        
    all_words = list(dictionary.keys())
    all_words.sort()
    return dictionary, all_words

In [10]:
dictionary, all_words = make_dictionary()

### 3. Dla każdego dokumentu j wyznacz wektor cech bag-of-words ${d_j}$ zawierający częstości występowania poszczególnych słów (termów) w tekście.

In [26]:
def get_bag_of_words(text, number_of_words):
    bag_of_word = [0] * number_of_words
    
    text_list = text.split()
    text_number_of_words = len(text_list)
    
    for word in text_list:
        index = all_words.index(word)
        bag_of_word[index] += 1
        
    for index, word in enumerate(all_words):
        bag_of_word[index] /= text_number_of_words
        
    return bag_of_word

### 4. Zbuduj rzadką macierz wektorów cech term-by-document matrix w której wektory cech ułożone są kolumnowo $A_{m×n} = [d_1|d_2| . . . |d_n]$ (m jest liczbą termów w słowniku, a n liczbą dokumentów)

In [27]:
def get_words_in_document(number):
    for index, count in enumerate(term_frequency[number]):
        if count > 0:
            print(all_words[index], end=" ")

In [36]:
def create_term_frequency():
    number_of_words = len(all_words)
    term_frequency = []
    
    files = sort_dir(os.listdir('data/'))
    
    for file in files:
        text = file_preprocess(file)
        if text is None: continue
            
        bag_of_word = get_bag_of_words(text, number_of_words)
            
        term_frequency.append(bag_of_word)
        print("Loading: " + file, end="\r")
                
    return np.array(term_frequency)

In [37]:
term_frequency = create_term_frequency()

### 5. Przetwórz wstępnie otrzymany zbiór danych mnożąc elementy bag-of-words przez inverse document frequency. Operacja ta pozwoli na redukcję znaczenia często występujących słów.
$$IDF(w) = log \frac{N}{n_w} = log \frac{N}{DF(t)}$$
### gdzie ${n_w}$ jest liczbą dokumentów, w których występuje słowo w, a N jest całkowitą liczbą dokumentów.

In [77]:
def get_inverse_document_frequency():
    N = len(term_frequency)
    inverse_document_frequency = []
    
    for index, word in enumerate(all_words):
        documents_with_word = 0
        for document in term_frequency:
            if document[index] > 0: 
                documents_with_word += 1 # Document Frequency

        inverse_document_frequency.append(math.log(N / documents_with_word)) # Inverse Document Frequency
    
    return np.array(inverse_document_frequency)

In [78]:
inverse_document_frequency = get_inverse_document_frequency()

In [103]:
TF_IDF = []
for document in term_frequency:
    TF_IDF.append(document * inverse_document_frequency) 
    
TF_IDF = np.array(TF_IDF)

### 6. Napisz program pozwalający na wprowadzenie zapytania (w postaci sekwencji słów) przekształcanego następnie do reprezentacji wektorowej q (bag-of-words). Program ma zwrócić k dokumentów najbardziej zbliżonych do podanego zapytania q. Użyj korelacji między wektorami jako miary podobieństwa

$$cos{θ_j} = \frac{q^{T}{d_j}}{||q||*||{d_j}||} = \frac{q^{T}A{e_j}}{||q||*||A{e_j}||}$$

In [68]:
def review_open(file_name):
    f = io.open('data/'+ str(file_name) + ".txt", encoding='windows-1254')
    text = f.read()
    f.close()
    print(text)

In [90]:
def do_query(query, k, method, print_review = True):
    number_of_words = len(all_words)
    text = preprocess(query)
    text_list = text.split()
    text = ""
    
    for word in text_list:
        if word in all_words:
            text += word + " "
            
    query_vector = np.array(get_bag_of_words(text, number_of_words))
    query_norm = np.linalg.norm(query_vector)
    result = []
    
    for index, document in enumerate(method):
        product = query_vector.T @ document
        divider = query_norm * np.linalg.norm(document)
        cos_theta = product / divider
        result.append((cos_theta, index))
    
    result.sort(key = lambda tup: tup[0], reverse=True)
    
    print("Search results for \"{}\": ".format(query), end = "")
    for document in result[:k]: print(document[1], end=" ")
        
    if print_review: 
        print("\n\nReview number:", result[0][1])
        review_open(result[0][1])
    print()    
        
do_query("fugitive sequel, thriller with harrison ford", 5, TF_IDF)
do_query("star wars phantom menace", 5, TF_IDF, False)

Search results for "fugitive sequel, thriller with harrison ford": 1138 673 929 653 732 

Review number: 1138
the sequel to the fugitive ( 1993 ) , u . s marshals is an average thriller using it's association with the fugitive just so it can make a few extra bucks . tommy lee jones returns to his role as chief deputy samuel gerard , the grizzly cop who was after harrison ford in the fugitive . this time , he's after fugitive mark sheridan ( snipes ) who the police think killed two fbi agents , but of course he's 

Search results for "star wars phantom menace": 822 1052 1313 227 290 


### 7. Zastosuj normalizację wektorów cech ${d_j}$ i wektora q, tak aby miały one długość 1. Użyj zmodyfikowanej miary podobieństwa otrzymując:
$$|q^{T}A| = [|cos{θ_1}|,|cos{θ_2}|,...,|cos{θ_n}|]$$

In [104]:
def get_query_list(query, k, method):
    number_of_words = len(all_words)
    text = preprocess(query)
    text_list = text.split()
    text = ""
    
    for word in text_list:
        if word in all_words:
            text += word + " "
            
    query_vector = np.array(get_bag_of_words(text, number_of_words))
    query_norm = np.linalg.norm(query_vector)
    result = []
    
    for index, document in enumerate(np.array(method)):
        product = query_vector.T @ document
        divider = query_norm * np.linalg.norm(document)
        cos_theta = product / divider
        result.append(cos_theta)
    
    return result

query_list = get_query_list("fugitive sequel, thriller with harrison ford", 5, TF_IDF)

### 8. W celu usunięcia szumu z macierzy A zastosuj SVD i low rank approximation otrzymując:
$$A≃{A_k}={U_k}{D_k}{V_k}^T=[{u_1}|...|{u_k}] 
\begin{bmatrix}{σ_1}&&\\&\ddots&\\&&{σ_k}\end{bmatrix}
\begin{bmatrix}{v_1}^T\\\vdots\\{v_k}^T\end{bmatrix} = \sum_{i=1}^{k} {σ_i}{u_i}{v_i}^T
$$

### oraz nową miarę podobieństwa
$$cos{θ_j} = \frac{q^{T}{A_k}{e_j}}{||q||*||{A_k}{e_j}||} $$

In [105]:
def get_low_rank_approximation(k):
    u, s, vt = scipy.sparse.linalg.svds(np.array(TF_IDF), k=k)
    return u @ np.diag(s) @ vt

In [106]:
low_rank_approximation = []
for k in range(5,101,5):
    print(k, end="\r")
    low_rank_approximation.append(get_low_rank_approximation(k)) 

### 9. Porównaj działanie programu bez usuwania szumu i z usuwaniem szumu. Dla jakiej wartości k wyniki wyszukiwania są najlepsze (subiektywnie). 

In [28]:
query = "fugitive sequel, thriller with harrison ford"
do_query(query,5, TF_IDF, False)

for index, document in enumerate(low_rank_approximation):
    print(index*5 + 5, end=":: ")
    do_query(query,5, document, False)

Search results for "fugitive sequel, thriller with harrison ford": 1138 673 929 653 732 
5:: Search results for "fugitive sequel, thriller with harrison ford": 174 1180 850 233 1303 
10:: Search results for "fugitive sequel, thriller with harrison ford": 1272 1155 886 1022 1028 
15:: Search results for "fugitive sequel, thriller with harrison ford": 1138 1311 1124 767 1028 
20:: Search results for "fugitive sequel, thriller with harrison ford": 1138 850 1028 827 1311 
25:: Search results for "fugitive sequel, thriller with harrison ford": 1138 850 184 1350 1079 
30:: Search results for "fugitive sequel, thriller with harrison ford": 1138 850 477 923 184 
35:: Search results for "fugitive sequel, thriller with harrison ford": 1138 923 516 850 767 
40:: Search results for "fugitive sequel, thriller with harrison ford": 1138 850 767 923 516 
45:: Search results for "fugitive sequel, thriller with harrison ford": 1138 850 516 282 767 
50:: Search results for "fugitive sequel, thriller with

Działanie programu bez usuwania szumu jest bardzo podobne do tego z usuwaniem (dla k>20). Gdy k jest mniejsze od 20 wyniki wydają się bardzo losowe. Uważam, że optymalna wartość k to 60. Wyniki są wtedy bardzo dokładne i algorytm znajduje dokumenty najbardziej zbliżone do wyszukiwania. 

### Zbadaj wpływ przekształcenia IDF na wyniki wyszukiwania.

Podmieniłem liste TF_IDF na term_frequency i sprawdziłem jak IDF wpływa na wynik wyszukania.

Wniosek: IDF ma ogromny wpływ na wyniki wyszukiwania, dzieki temu popularne slowa ("film", "starring", itp.) nie są tak bardzo premiowane w wyszukiwaniu. Co za tym idzie słowa kluczowe (rzadkie) mają wiekszy wpływ przy wyszukaniu konkretnej recenzji.